In [12]:
##importing necessary libraries

In [13]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np

In [15]:
##preprocessing the images

#reshaping the images because the images have 3 dimensions when neural networks expect 4 dimensions(batchsize, height, width, channels)
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')

# Normalize to [-1, 1] because gan uses tanh which has a range from -1,1
train_images = (train_images - 127.5) / 127.5

# Batch size
batch_size = 256
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(60000).batch(batch_size)

In [16]:
#building the generator
#generator will generate fake images to fool the discriminator

In [34]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

In [35]:
def generator():

    ##here the generator takes in the random noise vector and reshapes it high dimensions to form basis of fake images
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))

    ##ensures that the mean of the random vector is 0 and tsnadard deviation is 1
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    
    ##thid will convert the latent space into a fake grayscale image with dimensions 28*28 with -1,1 normalization range since gan uses tanh as it output layer activation function.
    model.add(layers.Reshape((7, 7, 256)))
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

In [36]:
##building the discriminator model.
##the discriminator will train itself so as to identify wheteher the image generated by the generator is fake or real
##the discriminator uses convolution neural networks in order to detect fae or real image

In [37]:
def discriminator():

    ##convolution layers
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    ##flattenening to reduce the high dimensional pooling vectors to 1D layer for cnn
    model.add(layers.Flatten())

    ##final detection whether the image is fake or real, if close to one real or else fake.
    model.add(layers.Dense(1))

    return model

In [38]:
##calculate loss

In [39]:

cross_entropy = tensorflow.keras.losses.BinaryCrossentropy(from_logits=True)

##discriminator loss measures how well the discriminator can detect fake and real images
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tensorflow.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tensorflow.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
##generator loss is determined by how well the genertao fools the discriminator
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers for both generator and discriminator
generator_optimizer = tensorflow.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tensorflow.keras.optimizers.Adam(1e-4)

generator = make_generator_model()
discriminator = make_discriminator_model()

NameError: name 'tensorflow' is not defined

In [40]:
##train model

In [41]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

random= tf.random.normal([num_examples_to_generate, noise_dim])

In [42]:
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    ##will keep track of operations to perform during back propogation
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    ##optimiser for both generator and discriminator
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [43]:
##training loop

In [1]:
def train(train_df, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)

        # Generate and save images after every epoch
        display.clear_output(wait=True)
        generate_and_save_images(generator, epoch + 1, seed)

        # here, we save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

    # Generate final images
    display.clear_output(wait=True)
    generate_and_save_images(generator, epochs, random)

In [2]:
def generate_and_save_images(model, epoch, test_img):
    predictions = model(test_img, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [3]:
for test_image in test_df:  # Assuming you have a test set
    enhanced_image = generator(test_image)
    plt.imshow((enhanced_image[0] * 127.5 + 127.5).numpy().astype(np.uint8))
    ##this will convert the image back to 0 to 255 which was normalized to -1, 1
    plt.show()

NameError: name 'test_df' is not defined